# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [1]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      Using setuptools (version 69.1.1).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\Box2D
      copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-312\Box2D
      copying library\Box2D\__init__.py -> build\lib.win-amd64-cpython-312\Box2D
      creating build\lib.win-amd64-cpython-312\Box2D\b2
      copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-cpython-312\Box2D\b2
      running build_ext
      building 'Box2D._Box2D' extension
      swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
      swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
      Box2D\Common\

## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [6]:
!pip install box2d
!pip install gymnasium[classic_control]


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 435.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.3 MB 1.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.3 MB 3.1 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.3 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 6.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached gymnasium-1.2.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)
Using cached gymnasium-1.2.0-py3-none-any.whl (944 kB)



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [10]:
import numpy as np

def random_agent_function(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00262156  1.4170023  -0.26555532  0.2703232   0.00304456  0.06015223
  0.          0.        ] -> Action: 2
Obs: [-0.00507488  1.4233803  -0.24916753  0.2834523   0.00683642  0.07584468
  0.          0.        ] -> Action: 1
Obs: [-0.0075963   1.429149   -0.25770447  0.25635517  0.01233647  0.11001129
  0.          0.        ] -> Action: 1
Obs: [-0.01020155  1.4343108  -0.2682313   0.22932853  0.01994456  0.15217595
  0.          0.        ] -> Action: 2
Obs: [-0.01297207  1.4401734  -0.28402588  0.2604557   0.02684418  0.13800527
  0.          0.        ] -> Action: 1
Obs: [-0.01583347  1.4454448  -0.29541773  0.23408753  0.03601994  0.18353207
  0.          0.        ] -> Action: 2
Obs: [-0.01856327  1.4509318  -0.28297502  0.2435946   0.04591576  0.19793451
  0.          0.        ] -> Action: 3
Obs: [-0.02119522  1.4558221  -0.27070472  0.21710238  0.05334343  0.14856738
  0.          0.        ] -> Action: 3
Obs: [-0.02374697  1.4601303  -0.26061326  0.19127797  0.0587298

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [11]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [12]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent_function)

Obs: [-0.00543947  1.416107   -0.55097866  0.23052248  0.00630981  0.12480482
  0.          0.        ] -> Action: 0
Obs: [-0.01087942  1.4207165  -0.55023265  0.20482473  0.01247698  0.12335543
  0.          0.        ] -> Action: 0
Obs: [-0.01631956  1.4247271  -0.5502522   0.17818493  0.01864046  0.12328124
  0.          0.        ] -> Action: 0
Obs: [-0.02175989  1.4281381  -0.5502704   0.15151136  0.02480317  0.12326562
  0.          0.        ] -> Action: 0
Obs: [-0.02720051  1.4309498  -0.5502883   0.12484017  0.0309649   0.12324611
  0.          0.        ] -> Action: 0
Obs: [-0.03264131  1.4331617  -0.55030656  0.09816916  0.03712565  0.12322627
  0.          0.        ] -> Action: 0
Obs: [-0.03808231  1.4347743  -0.5503246   0.07149806  0.04328543  0.12320682
  0.          0.        ] -> Action: 0
Obs: [-0.0435236   1.4357873  -0.5503427   0.04482675  0.04944422  0.1231872
  0.          0.        ] -> Action: 0
Obs: [-0.04896498  1.4362007  -0.55036056  0.0181553   0.05560204

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [13]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [16]:
import gymnasium as gym
import numpy as np
import time

def better_reflex_agent_stable(obs):
    x, y, vx, vy, angle, ang_vel, left_leg, right_leg = obs
    
    # --- Hệ số ổn định ---
    Kp_pos = 0.5      # tác động theo vị trí x
    Kd_vel = 0.3      # tác động theo vận tốc vx
    angle_threshold = 0.05
    y_slowdown = 0.5
    vy_thresh = -0.3
    vx_near_ground = 0.05
    
    # --- Góc mục tiêu ---
    angle_target = Kp_pos * x + Kd_vel * vx
    angle_diff = angle - angle_target
    
    # --- Thrusters trái/phải để cân bằng ---
    if angle_diff > angle_threshold:
        return 3  # fire right thruster
    if angle_diff < -angle_threshold:
        return 1  # fire left thruster
    
    # --- Giảm tốc độ ngang gần đất ---
    if y < y_slowdown:
        if vx > vx_near_ground:
            return 1  # fire left thruster
        if vx < -vx_near_ground:
            return 3  # fire right thruster
    
    # --- Main engine để giảm tốc rơi ---
    if vy < vy_thresh:
        return 2  # fire main engine
    
    return 0  # mặc định không làm gì

# --- Hàm demo mô phỏng ---
def show_demo_stable(episodes=3):
    env = gym.make("LunarLander-v3", render_mode="human")
    for ep in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = better_reflex_agent_stable(obs)
            obs, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            done = terminated or truncated
            time.sleep(0.02)
        print(f"Episode {ep+1} reward = {total_reward:.2f}")
    env.close()

# --- Hàm đánh giá 100 episode ---
def evaluate_stable(episodes=100):
    env = gym.make("LunarLander-v3", render_mode=None)
    rewards = []
    success_count = 0
    last_obs = None
    for ep in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = better_reflex_agent_stable(obs)
            obs, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            done = terminated or truncated
        rewards.append(total_reward)
        last_obs = obs
        if total_reward >= 100:  # reward ≥100 → hạ cánh thành công
            success_count += 1
    env.close()
    
    avg = np.mean(rewards)
    med = np.median(rewards)
    print(f"Average reward: {avg:.2f}")
    print(f"Median reward: {med:.2f}")
    print(f"Success count (reward >= 100): {success_count}/{episodes}")
    return avg, med, success_count, last_obs

# --- Chạy demo ---
show_demo_stable(3)        # xem mô phỏng 3 lần
evaluate_stable(100)       # đánh giá 100 episode


Episode 1 reward = -45.27
Episode 2 reward = -15.26
Episode 3 reward = -33.99
Average reward: -52.46
Median reward: -50.85
Success count (reward >= 100): 2/100


(-52.45932971942354,
 -50.8539724806988,
 2,
 array([ 0.30467206,  0.04271766, -0.13387224, -1.1681648 ,  0.4340846 ,
         5.8873377 ,  1.        ,  1.        ], dtype=float32))